In [13]:
import pandas as pd

df = pd.read_json('./outputMistral.jsonl',lines=True)

df_train = df.sample(frac=0.995, random_state=200)

df_eval = df.drop(df_train.index)

In [14]:
df_train.to_json('./training_dataset/df_train.jsonl', orient='records', lines=True)

df_eval.to_json('./training_dataset/df_eval.jsonl', orient='records', lines=True)

In [12]:
import json

def transform_json(input_json):
    transformed_json = {
        "messages": [
            {
                "role": "user",
                "content": input_json["instruction"]
            },
            {
                "role": "assistant",
                "content": input_json["cot_answer"]
            }
        ]
    }
    return transformed_json

def process_jsonl(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            input_json = json.loads(line)
            transformed_json = transform_json(input_json)
            json.dump(transformed_json, outfile)
            outfile.write('\n')

input_file = './output.jsonl'
output_file = './outputMistral.jsonl'

process_jsonl(input_file, output_file)


In [16]:
!wget https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py

/Users/vedantsasane/.zshenv:.:1: no such file or directory: /Users/vedantsasane/.cargo/env
--2024-07-30 23:32:14--  https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3381 (3.3K) [text/plain]
Saving to: ‘reformat_data.py’

reformat_data.py    100%[===================>]   3.30K  --.-KB/s    in 0.03s   

2024-07-30 23:32:16 (98.7 KB/s) - ‘reformat_data.py’ saved [3381/3381]



In [17]:
!python reformat_data.py './training_dataset/df_train.jsonl'

/Users/vedantsasane/.zshenv:.:1: no such file or directory: /Users/vedantsasane/.cargo/env


In [18]:
!python reformat_data.py './training_dataset/df_eval.jsonl'

/Users/vedantsasane/.zshenv:.:1: no such file or directory: /Users/vedantsasane/.cargo/env


In [19]:
df_train.iloc[3674]['messages']

[{'role': 'user',
  'content': '<DOCUMENT>No. Stds/SP/(Contaminants)/Notification -1/FSSAI -2018, dated 7th \nAugust,  2020. </DOCUMENT>\n<DOCUMENT>(II) Fats derived from animal  \ntissues  \n(III) Milk  0.01  \n24. Danofloxacin  Cattle  \nMuscle  0.2 \nLiver  0.4 \nKidney  0.4 \nFat 0.1 \nPig \nMuscle  0.1 \nLiver  0.05  \nKidney  0.2 \nFat 0.1 \nChicken  \nMuscle  0.2 \nLiver  0.4 \nKidney  0.4 \nFat 0.1 \n25. Enrofloxacin  (I) All edible  animal  tissues  0.01  Version  –VI (27.01.2022 )  S.</DOCUMENT>\n<DOCUMENT>Name of the antibiotics and  \nveterinary  drugs  Food  Tolerance  limit  \n(mg/Kg)  \n(1) (2) (3) (4) \n1. Ampicillin  (I) All edible  animal  tissues  \n(II) Fats  derived  from  \nanimal  tissues  \n(III) Milk  0.01  \n2. Cloxacillin  (I) All edible  animal  tissues  \n(II) Fats  derived  from  \nanimal  tissues  \n(III) Milk  0.01  \n3. Colistin  Cattle  \nFat 0.15  \nMuscle  0.15  \nKidney  0.2 \nLiver  0.15  \nMilk  0.05  \nPig \nMuscle  0.15  \nFat 0.15  \nLiver  0.1

In [21]:
import os
from mistralai.client import MistralClient

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("./training_dataset/complyai_train.jsonl", "rb") as f:
    complyai_train = client.files.create(file=("complyai_train.jsonl", f))
with open("./training_dataset/complyai_eval.jsonl", "rb") as f:
    complyai_eval = client.files.create(file=("complyai_eval.jsonl", f))

In [22]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [23]:
pprint(complyai_train)

{
    "id": "2fa3cae9-5754-41fb-a681-e3202bcb5b13",
    "object": "file",
    "bytes": 21637401,
    "created_at": 1722365123,
    "filename": "complyai_train.jsonl",
    "purpose": "fine-tune"
}


In [24]:
pprint(complyai_eval)

{
    "id": "4793c647-c842-443a-b21c-52859199d117",
    "object": "file",
    "bytes": 100932,
    "created_at": 1722365124,
    "filename": "complyai_eval.jsonl",
    "purpose": "fine-tune"
}


In [29]:
from mistralai.models.jobs import WandbIntegrationIn,TrainingParameters

wandb_api_key = "2b9312973a66d517666e583df1bfcd50898fd8ca"
print(wandb_api_key)
created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[complyai_train.id],
    validation_files=[complyai_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=100,
        learning_rate=0.0001,
    ),
    integrations=[
        WandbIntegrationIn(
            project="complyai_ft",
            run_name="ft_test",
            api_key=wandb_api_key,
        ).dict()
    ],
)

2b9312973a66d517666e583df1bfcd50898fd8ca


In [33]:
print(created_jobs)

id='5b3db1fc-a667-43e6-9331-0e1e6ccb67a3' hyperparameters=TrainingParameters(training_steps=100, learning_rate=0.0001) fine_tuned_model=None model='open-mistral-7b' status='QUEUED' job_type='FT' created_at=1722366451 modified_at=1722366451 training_files=['2fa3cae9-5754-41fb-a681-e3202bcb5b13'] validation_files=['4793c647-c842-443a-b21c-52859199d117'] object='job' integrations=[WandbIntegration(type='wandb', project='complyai_ft', name=None, run_name='ft_test')]


In [34]:
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)

{
    "id": "5b3db1fc-a667-43e6-9331-0e1e6ccb67a3",
    "hyperparameters": {
        "training_steps": 100,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": "ft:open-mistral-7b:fe3c5ae7:20240730:5b3db1fc",
    "model": "open-mistral-7b",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1722366451,
    "modified_at": 1722367082,
    "training_files": [
        "2fa3cae9-5754-41fb-a681-e3202bcb5b13"
    ],
    "validation_files": [
        "4793c647-c842-443a-b21c-52859199d117"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "complyai_ft",
            "name": null,
            "run_name": "ft_test"
        }
    ],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "SUCCESS"
            },
            "created_at": 1722367082
        },
        {
            "name": "status-updated",
            "data": {
                "status": "RUN

In [35]:
from mistralai.models.chat_completion import ChatMessage

chat_response = client.chat(
    model=retrieved_jobs.fine_tuned_model,
    messages=[ChatMessage(role='user', content='How can i get a License for a food business')]
)

In [36]:
pprint(chat_response)

{
    "id": "bd79c35e37e3430190eb091805659791",
    "object": "chat.completion",
    "created": 1722367621,
    "model": "ft:open-mistral-7b:fe3c5ae7:20240730:5b3db1fc",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "To get a license for a food business, follow these steps:\n\n1. **Identify the Requirements**: Understand what is necessary for a food business license in your area. This may include specific types of licenses, inspections, permits, and other requirements.\n\n2. **Choose the Right License**: Determine which type of license is appropriate for your food business. This could be a food service license, food handling license, or another type of license.\n\n3. **Locate the Regulatory Body**: Identify the regulatory body responsible for issuing food business licenses in your area. This is typically a local health department or food safety authority.\n\n4. **Gather Necessary Documents**: 